In [1]:
import os
import json
import pandas as pd
import traceback

In [5]:
from langchain.chat_models import ChatOpenAI

In [13]:
from dotenv import load_dotenv
load_dotenv() #take environment variables from .env

True

In [14]:
KEY=os.environ.get("OPENAI_API_KEY")
GPT_MODEL=os.environ.get("GPT_MODEL")

gpt-3.5-turbo-16k


In [15]:
llm=ChatOpenAI(openai_api_key=KEY, model=GPT_MODEL, temperature=0.5)

In [17]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [18]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [19]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [20]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [21]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [22]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [23]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [24]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [25]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [26]:
file_path = r"C:\Users\jacky\mcqgen\data.txt"

In [28]:
with open(file_path, "r") as file:
    TEXT = file.read()

In [29]:
TEXT

"Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]\n\nMachine learning approaches have been applied to many fields including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] ML is known in its application across business problems under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.\n\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsuper

In [30]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [31]:
NUMBER=5 
SUBJECT="data science"
TONE="simple"

In [32]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

C:\Users\jacky\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]

Machine learning approaches have been applied to many fields including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] ML is known in its application across business problems under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data min

In [33]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1519
Prompt Tokens:904
Completion Tokens:615
Total Cost:0.0051719999999999995


In [36]:
response

{'text': "Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]\n\nMachine learning approaches have been applied to many fields including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] ML is known in its application across business problems under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.\n\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) throug

In [40]:
quiz=response.get("quiz")
print(quiz)

{"1": {"mcq": "Which field is machine learning a part of?", "options": {"a": "Artificial intelligence", "b": "Data science", "c": "Computer programming", "d": "Robotics"}, "correct": "a"}, "2": {"mcq": "What is the goal of machine learning?", "options": {"a": "To develop statistical algorithms", "b": "To study artificial neural networks", "c": "To perform tasks without explicit instructions", "d": "To explore data through unsupervised learning"}, "correct": "c"}, "3": {"mcq": "Which field has machine learning been applied to?", "options": {"a": "Natural language processing", "b": "Astronomy", "c": "Geology", "d": "Philosophy"}, "correct": "a"}, "4": {"mcq": "What provides the mathematical foundations of machine learning?", "options": {"a": "Data mining", "b": "Mathematical optimization", "c": "Predictive analytics", "d": "Computational statistics"}, "correct": "b"}, "5": {"mcq": "What framework describes machine learning from a theoretical viewpoint?", "options": {"a": "Parallel comput

In [43]:
quiz=json.loads(quiz)

In [44]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [45]:
quiz_table_data

[{'MCQ': 'Which field is machine learning a part of?',
  'Choices': 'a: Artificial intelligence | b: Data science | c: Computer programming | d: Robotics',
  'Correct': 'a'},
 {'MCQ': 'What is the goal of machine learning?',
  'Choices': 'a: To develop statistical algorithms | b: To study artificial neural networks | c: To perform tasks without explicit instructions | d: To explore data through unsupervised learning',
  'Correct': 'c'},
 {'MCQ': 'Which field has machine learning been applied to?',
  'Choices': 'a: Natural language processing | b: Astronomy | c: Geology | d: Philosophy',
  'Correct': 'a'},
 {'MCQ': 'What provides the mathematical foundations of machine learning?',
  'Choices': 'a: Data mining | b: Mathematical optimization | c: Predictive analytics | d: Computational statistics',
  'Correct': 'b'},
 {'MCQ': 'What framework describes machine learning from a theoretical viewpoint?',
  'Choices': 'a: Parallel computing | b: Approximate reasoning | c: Exploratory data analy

In [47]:
quiz=pd.DataFrame(quiz_table_data)

In [48]:
quiz.to_csv("machinelearning.csv",index=False)